In [2]:
import inspect
import random
from weakref import WeakKeyDictionary

In [239]:
class Symbol(object):
    def __init__(self, values):
        self.parent = None
        self.InitVals(values)
        
    def InitVals(self, values):
        if inspect.isclass(values):
            self.values = []
            for subclass in values.__subclasses__():
                self.values.append(subclass())
        else:
            self.values = values
        self._value = None
        # update my attributes
        self.CalcAttributes()
    
    def __getattr__(self, attr):
        if attr == 'val':
            if self._value:
                return self._value
            else:
                if len(self.values) == 1:
                    return self.values[0]
                else:
                    return self.values
        raise AttributeError
    
    def Ref(self):
        if self._value:
            return self._value
        else:
            return self.Collapse()
    
    def Collapse(self):
        self._value = random.choice(self.values)
        # propagate upwards
        if self.parent:
            self.parent.Restrict(self, self._value)
            
        # update my attributes?
        self.CalcAttributes()
        return self._value
    
    def CalcAttributes(self):
        asets = {}
        for obj in self.values:
            attrs = [i for i in dir(obj) if i[0] != '_' and not callable(getattr(obj, i))]
#             print(attrs)
            for attr in attrs:
                if attr not in asets:
                    asets[attr] = []
                val = obj.__getattribute__(attr)
                if isinstance(val, Symbol):
                    asets[attr].extend(val.values)
                else:
                    asets[attr].append(val)
        for key, vals in asets.items():
#             if len(vals) == 1:
#                 setattr(self, key, vals[0])
#             else:
                symbol = Symbol(vals)
                symbol.parent = self
                setattr(self, key, symbol)

    def Restrict(self, child, newval):
        # change content of values to be consistent with new value for child
        print(str(self)+" restricting")
        for key, val in self.__dict__.items():
            if val == child:
                print(key, ' = ', newval)
                attr = key
                break
        new_values = []
        for obj in self.values:
            print('checking object '+str(obj))
            # if not has attr, next
            if not hasattr(obj, attr):
                continue
            
            # if is obj.attr is symbol, check that obj.attr.values contains newattr
            if isinstance(obj, Symbol):
                if newval in obj.values:
                    # potential problem with instances vs classes
                    new_values.append(obj)
                    continue
            else:
                # if obj.attr is not symbol, check that obj.attr is newval
                if getattr(obj, attr) == newval:
                    new_values.append(obj)
                    continue
        print(new_values)
        if len(new_values) == 0:
            raise Exception('empty restriction!')
        self.values = new_values
        if len(new_values) == 1:
            print('restriction collapse')
            self.Collapse()
            

In [240]:
class Symbolic(object):
    def __init__(self):
        self.symbols = WeakKeyDictionary()
 
    def __get__(self, instance_obj, objtype):
#         return self.symbols[instance_obj].Ref()
        return self.symbols[instance_obj]
 
    def __set__(self, instance, values):
        self.symbols[instance] = Symbol(values)
 
    def __delete__(self, instance):
        del self.symbols[instance]

        
class Fruit(object):
    color = Symbolic()
    def __init__(self, name):
        self.name = name
    def method(self):
        print('method man')

class Apple(Fruit):
    def __init__(self):
        self.name = 'Apple'
        self.color = ['red', 'green']

class Pear(Fruit):
    def __init__(self):
        self.name = 'Pear'
        self.color = ['red', 'yellow']

        
class State(object):
    fruit = Symbolic()
    def __init__(self):
        self.fruit = Fruit

In [241]:
state = State()
print(state.fruit)
print(state.fruit.color.val)

['red', 'green', 'red', 'yellow']


In [242]:
state.fruit.color.Ref()

<__main__.Symbol object at 0x1105e99e8> restricting
color  =  green
checking object <__main__.Apple object at 0x1105e9ef0>
checking object <__main__.Pear object at 0x1105e9b00>
[]


Exception: empty restriction!

In [246]:
x = state.fruit.values[0]

In [248]:
hasattr(x, 'color')

True

In [249]:
getattr(x, 'color')